If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [1]:
# !pip install pyomo==6.4.1
# !apt install glpk-utils
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

# Assignment 1 - Linear Programming (LP)

## 4.

Consider two generating units supplying at least the system demand P_total,D=650 MW for the coming hour (extra power above the system demand will be used for storage, which is assumed to be unlimited). The unit cost function is characterized by the parameters provided in the table below:

| Unit | Generation costs (EUR/MWh) | P_min,G (MW) | P_max,G (MW) |
|:---|---:|---:|---:|
| 1 | 35 | 0 | 600 |
| 2 | 55 | 100 | 300 |

Tasks 4.a) through 4.c) are solved on paper.

### 4.d)

Solve the problem in Python and specifically examine the shadow prices delivered by Pyomo for the optimal solution. Compare your solutions here with your answers above.

### 4.e)

Formulate and solve the dual problem in Python to calculate the shadow prices yourself. What do you observe regarding the solution of the primal and the dual problem?

<div class="alert alert-block alert-info">

💡 You do not need to call every model `model`. For example, if you call the dual model `dual_model` instead of `model`, the original (primal) model is still available as well, if you want to analyse it further in this same Python session.

</div>

## 5

### 5.a)

### 5.b)